# LF download (no processing)
- build out routines to download LF OOI data without any processing

In [103]:
import kaooi
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from dask.distributed import Client

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [104]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 32,Total memory: 62.79 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37821,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 32
Started: Just now,Total memory: 62.79 GiB
Comm: tcp://127.0.0.1:44431,Total threads: 4
Dashboard: http://127.0.0.1:42685/status,Memory: 7.85 GiB
Nanny: tcp://127.0.0.1:41975,


In [102]:
fn = '/datadrive/kauai/transmissions/ooi_lf/*.nc'
ds = xr.open_mfdataset(fn)

### look at clipping

In [ ]:
fig = plt.figure(figsize=(13,5))

for k, node in enumerate(ds.keys()):
    plt.subplot(2,3,k+1)
    ds[node][:,0].plot()
    mean = ds[node][:,0].mean()
    std = 4*ds[node][:,0].std()
    plt.plot([0,900],[mean + std, mean+std], 'k--')
    plt.plot([0,900],[mean - std, mean-std], 'k--')
    plt.ylim([mean - std*1.5, mean + std*1.5])

## Play around with the processing

In [113]:
rep = kaooi.construct_replica(sampling_rate=200)

In [123]:
ds_coords = ds.assign_coords({'time':np.arange(0, 1434928/200, 1/200)}).drop_vars(['longtime', 'shorttime'])

In [124]:
ds_match = kaooi.match_filter(ds_coords, dim='time', sampling_rate=200, length='2H')

/home/jhrag/anaconda3/envs/ooi_dev_archive/lib/python3.10/site-packages/distributed/worker.py:2936: UserWarning: Large object of size 10.95 MiB detected in task graph: 
  ([('time',), <xarray.IndexVariable 'time' (time: 1 ... 35e+03]), {}],)
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  warnings.warn(
/home/jhrag/anaconda3/envs/ooi_dev_archive/lib/python3.10/site-packages/scipy/signal/_signaltools.py:240: RuntimeWarning: Use of fft convolution on input with NAN or inf results in NAN or inf output. Consider using method='direct' instead.
  return convolve(in1, _reverse_and_conj(in2), mode, method)
/home/jhrag/anaconda3/envs/ooi_dev_archive/lib/python3.10/site-packages/scipy/signal/_signaltools.py:240: RuntimeWarning: Use of fft convolution on input with 

KeyboardInterrupt: 

2023-11-09 20:59:03,210 - distributed.nanny - WARNING - Restarting worker
2023-11-09 20:59:03,213 - distributed.nanny - WARNING - Restarting worker
2023-11-09 20:59:03,215 - distributed.nanny - WARNING - Restarting worker
2023-11-09 20:59:03,218 - distributed.nanny - WARNING - Restarting worker
2023-11-09 20:59:03,225 - distributed.nanny - WARNING - Restarting worker
2023-11-09 20:59:03,229 - distributed.nanny - WARNING - Restarting worker
2023-11-09 20:59:03,232 - distributed.nanny - WARNING - Restarting worker
2023-11-09 20:59:03,235 - distributed.nanny - WARNING - Restarting worker


In [ ]:
ds_match